In [ ]:
# Patterns detection

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

In [21]:
df = pd.read_csv("BTCUSD.csv")
#Check if NA values are in data
# df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,...,BBW,MACD,RSI_signal,BB_signal,MACD_signal,MA_signal,Engulfing,Consolidation,Breakout,ATR
0,2017-03-28 03:00:00,1041.319,1041.320,1041.220,1041.222,68,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
1,2017-03-28 04:00:00,1041.222,1042.778,1041.220,1042.000,185,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
2,2017-03-28 05:00:00,1043.000,1043.000,1042.007,1042.007,32,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
3,2017-03-28 06:00:00,1042.989,1042.989,1042.000,1042.003,50,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
4,2017-03-28 07:00:00,1043.000,1043.000,1041.504,1042.007,162,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53629,2023-07-08 09:00:00,30260.738,30271.838,30209.738,30235.738,820,30293.99028,30564.08469,49.037318,30394.861410,...,260.166820,-10.482079,0,0,-1,-1,0,0,0,80.315071
53630,2023-07-08 10:00:00,30235.738,30240.938,30204.838,30209.338,573,30271.69028,30561.89719,47.285800,30373.158793,...,235.261587,-11.954919,0,0,-1,-1,0,0,0,76.079357
53631,2023-07-08 11:00:00,30209.338,30216.738,30141.638,30182.738,1043,30254.08428,30559.14519,45.521522,30354.172892,...,216.439784,-15.094552,0,0,-1,-1,0,0,0,75.150786
53632,2023-07-08 12:00:00,30184.038,30198.738,30131.338,30153.538,950,30232.73628,30555.57669,43.598460,30342.690476,...,212.224952,-19.711706,0,0,-1,-1,0,0,0,75.443643


In [22]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,3,3), axis=1)

## Pivot Candles Vizualisation

In [23]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [25]:

dfpl = df[48000:50000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="Blue"),
                name="head")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [15]:

backcandles = 20

candleid = 4696

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].high)
        xxmax = np.append(xxmax, i) # df.iloc[i].name
        
#slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
#slmax, intercmax = np.polyfit(xxmax, maxim,1)

slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

print(rmin, rmax)

dfpl = df[candleid-backcandles-10:candleid+backcandles+10]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="Blue"),
                name="head")

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
#adjintercmin = df.low.loc[candleid-backcandles:candleid].min() - slmin*df.low.iloc[candleid-backcandles:candleid].idxmin()
#adjintercmax = df.high.loc[candleid-backcandles:candleid].max() - slmax*df.high.iloc[candleid-backcandles:candleid].idxmax()

xxmin = np.append(xxmin, xxmin[-1]+15)
xxmax = np.append(xxmax, xxmax[-1]+15)
#fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
#fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))

fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

0.0 0.0


In [13]:

backcandles= 20

for candleid in range(11000, len(df)):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
    
    if (xxmax.size <3 and xxmin.size <3) or xxmax.size==0 or xxmin.size==0:
        continue
    
    #slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
    #slmax, intercmax = np.polyfit(xxmax, maxim,1)

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)
        
    #if abs(rmax)>=0.7 and abs(rmin)>=0.7 and abs(slmin)<=0.00001 and slmax<-0.0001:
    #if abs(rmax)>=0.7 and abs(rmin)>=0.7 and slmin>=0.0001 and abs(slmax)<=0.00001:
    if abs(rmax)>=0.9 and abs(rmin)>=0.9 and slmin>=0.0001 and slmax<=-0.0001:
        print(rmin, rmax, candleid)
        break
        
    if candleid % 1000 == 0:
        print(candleid)

11000
0.997176464952738 -1.0 11012
